# CMPE 258 Term Project - Breast Cancer Diagnosis Using Deep Transfer Learning

## Team AWS Members - Abhishek Bais, Wasae Qureshi, Subarna Chowdhury Soma 

## Problem Description:
### Context  
Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. To assign an aggressiveness grade to a whole mount sample, pathologists typically focus on the regions which contain the IDC. As a result, one of the common pre-processing steps for automatic aggressiveness grading is to delineate the exact regions of IDC inside of a whole mount slide.

### Content  
The original dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. From that, 277,524 patches of size 50 x 50 were extracted (198,738 IDC negative and 78,786 IDC positive). Each patch’s file name is of the format: uxXyYclassC.png — > example 10253idx5x1351y1101class0.png . Where u is the patient ID (10253idx5), X is the x-coordinate of where this patch was cropped from, Y is the y-coordinate of where this patch was cropped from, and C indicates the class where 0 is non-IDC and 1 is IDC.

### Acknowledgements  
1. The original files are located here: http://gleason.case.edu/webdata/jpi-dl-tutorial/IDC_regular_ps50_idx5.zip  

### Citation  
1. https://www.ncbi.nlm.nih.gov/pubmed/27563488 
2. http://spie.org/Publications/Proceedings/Paper/10.1117/12.2043872  

### Inspiration  
Breast cancer is the most common form of cancer in women, and invasive ductal carcinoma (IDC) is the most common form of breast cancer. Accurately identifying and categorizing breast cancer subtypes is an important clinical task, and automated methods can be used to save time and reduce error.

# Check if GPU/TPU available

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# Import data read, processing libraries
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import cv2
import imblearn
import inspect
import random
import subprocess
import tempfile
import json
import requests
from tqdm import tqdm
from io import BytesIO
from matplotlib.image import imread
import warnings
import tempfile
import sys
import shutil
import numpy as np
from numpy.random import shuffle, seed
import itertools

# Import visualization libraries
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Import tf, keras libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras import layers

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D,  LSTM, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.applications import VGG16
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import ReduceLROnPlateau

In [ ]:
%load_ext tensorboard

# 1.0. Data ingestion
### Data description: 
There are a total of 555,048 patient images in the Kaggle breast cancer 'breast-histopathology-images dataset'. 

### These contain
1. 397,646 images of patients with Benign/ No Cancer
2. 157,572 images of patients with Malignant/ Have Cancer

The images are obtained from the [breast cancer image dataset](https://www.kaggle.com/paultimothymooney/breast-histopathology-images) that can be found here.


## 1.1. Setup for Kaggle API first use

In [ ]:
!mkdir /root/.kaggle

In [ ]:
!mkdir /content/.kaggle

In [ ]:
import json
#token = {“username”:”YOUR-USER-NAME”,”key”:”SOME-VERY-LONG-STRING”}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s paultimothymooney/breast-histopathology-images

## 1.2. Read in the kaggle breast cancer dataset

In [ ]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images -p /kaggle/input/breast-histopathology-images

In [ ]:
!ls /kaggle/input/breast-histopathology-images

In [ ]:
# unzip
import zipfile
def unzip_images():
  with zipfile.ZipFile('/kaggle/input/breast-histopathology-images/breast-histopathology-images.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/input/breast-histopathology-images')

In [ ]:
unzip_images()

# 2.0. Data Pre-processing

## 2.1. Read in the breast cancer image file paths

In [ ]:
# Inspect a few unzipped images
images  = glob('/kaggle/input/breast-histopathology-images/**/*.png', recursive=True)
print(len(images))
for filename in tqdm(images[0:10]):
    print(filename)

## 2.2. Create labels from patient images
1. Benign/ No Cancer  - image files ending with class0.png
2. Malignant/ Have Cancer - image files ending with class1.png

In [ ]:
# Create labels
#  file names ending with class0.png are of patients without cancer (benign)
# file names ending with class1.png are of patients with cancer (malignant)

benign = [] 
malignant = [] 
def create_labels(images): 
  for filename in images:
    if filename.endswith("class0.png"):
         benign.append(filename)
    else:
        malignant.append(filename)
  print("Benign/ No Cancer", len(benign))
  print("Malignant/ Have Cancer", len(malignant))

In [ ]:
create_labels(images)

## 2.3. Inspect the benign, malignant class distribution

In [ ]:
# inspect the class distribution
df_benign = pd.DataFrame(benign, columns=['image'])
df_malignant = pd.DataFrame(malignant, columns=['image'])
df_benign['label'] = 0
df_malignant['label'] = 1
print("Benign/ No Cancer images sample", df_benign.shape)
print("Malignant/ Have Cancer", df_malignant.shape)

## 2.4. Report the class distributions

In [ ]:
# Report the distribution of benign vs malignant images
df_data = pd.concat([df_benign, df_malignant], axis=0).reset_index(drop=True)
plt.figure(figsize=(15,5), edgecolor='b')
df_data['label'].value_counts().plot(kind="barh", color='peru')
plt.title('Benign vs Malignant image distribution')

## 2.5. Resolve class imbalance by random downsampling majority class
### Pick 5000 images to avoid memory overhead of loading images
a. 2500 benign images  
b. 2500 malignant images  

In [ ]:
# Random downsample majority class, here benign to (total = ~150000)
sample_size = 2500
df_benign = df_benign.sample(sample_size, random_state=101)
df_malignant = df_malignant.sample(sample_size, random_state=101)

# concatenate the benign with malignant patient images
df_data = pd.concat([df_benign, df_malignant], axis=0).reset_index(drop=True)

# Report the benign vs malignant class distributions
df_data['label'].value_counts()

## 2.6. Inspect the class distributions after balancing

In [ ]:
# Report the distribution of benign vs malignant images
df_data = pd.concat([df_benign, df_malignant], axis=0).reset_index(drop=True)
plt.figure(figsize=(15,5), edgecolor='b')
df_data['label'].value_counts().plot(kind="barh", color='peru')
plt.title('Benign vs Malignant image distribution')

In [ ]:
df_data.head()

## 2.7. Load images from file paths

In [ ]:
# Resize images to 224, 224 to be compatible with pre-trained CNN models
def create_image_arrays(df, label):
  images_array = []
  for i in range(len(df)) :
    i = df.iloc[i].image
    if i.endswith('.png'):
      image = cv2.imread(i, cv2.IMREAD_COLOR)
      image_resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LINEAR) 
      images_array.append([image_resized, label])
  return images_array

In [ ]:
benign_images_array = create_image_arrays(df_benign, 0)
malignant_images_array = create_image_arrays(df_malignant, 1)
all_images = np.concatenate((benign_images_array, malignant_images_array))

In [ ]:
print(all_images.shape)

## 2.8. Normalize, Reshape image to 224, 224, 3 for different pre-trained CNN models


In [ ]:
# Reshape all images
X = []
y = []

for features,label in all_images:
    X.append(features)
    y.append(label)
print(len(X))
print(len(y))

X = np.array(X).reshape(-1, 224, 224, 3)
X = X/255.0
print(X.shape)

## 2.9. Create train, validation, test splits

### 2.9.1. Create 80/20 train/test split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=101)

### 2.9.2. Create 75/25 train/val split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=101)

### 2.9.3. 1-hot encode labels

In [ ]:
y_train = to_categorical(Y_train, num_classes = 2)
y_val = to_categorical(Y_val, num_classes = 2)
y_test = to_categorical(Y_test, num_classes = 2)

### 2.9.4. Examine input shapes for train, val, test images

In [ ]:
# print shapes of image samples in train, val, test
print("Shape of training images", X_train.shape)
print("Shape of validation images", X_val.shape)
print("Shape of test images", X_test.shape)

print("Shape of training labels", y_train.shape)
print("Shape of validation labels", y_val.shape)
print("Shape of test labels", y_test.shape)

# 3.0. Exploratory Data Analysis

In [ ]:
# View the data
def visualize_N_elems_of_dataset(X, y, row, col, name):
  print("Visualizing the " + name + " dataset.")
  fig = plt.figure(figsize=(15,10))

  # Plot row * col images of the dataset
  for image in range(1, col*row+1):
    ax = fig.add_subplot(row, col, image)
    if y[image] == 1:
        ax.title.set_text('Benign')
    else:
        ax.title.set_text('Malignant')
    ax.imshow(X[image], cmap='Accent')

## 3.1. View some images from training dataset

In [ ]:
# Explore some benign, malignant cancer patient images from training data
row = 2
col = 4
visualize_N_elems_of_dataset(X_train, Y_train, row, col, "train")

## 3.2. View some images from validation dataset

In [ ]:
# Explore some benign, malignant cancer patient images from validation data
row = 2
col = 4
visualize_N_elems_of_dataset(X_val, Y_val, row, col, "validation")

## 3.3 View some images from test dataset

In [ ]:
# Explore some benign, malignant cancer patient images from test data
row = 2
col = 4
visualize_N_elems_of_dataset(X_test, Y_test, row, col, "test")

# 4.0. Configure for Classification

## 4.1. Get pre-trained CNN models to evaluate
1. DenseNet201
2. InceptionResNetV2
3. NASNetMobile
4. ResNet50V2
5. ResNet152V2
6. VGG16

These will be evaluated against various metrics, best picked

In [ ]:
# inspect available pre-trained models in keras
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}
for name, model in tqdm(model_dictionary.items()):
  print(name)

In [ ]:
# check if model in models of interest
def models_to_evaluate_contains(name):
  contains = False
  if (("ResNet50V2" == name) or ("ResNet152V2" == name) or ("NASNetMobile" == name) or 
      ("VGG16" == name) or ("InceptionResNetV2" == name) or ("DenseNet201" == name) or ("Xception" == name)):
    contains = True
  return contains

# get pre-trained cnn feature models of interest
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}
my_pretrained_cnn_feature_models = {}
for name, model in tqdm(model_dictionary.items()):
  if models_to_evaluate_contains(name):
    my_pretrained_cnn_feature_models[name] = model

# print interesting models found to evaluate
for name, model in my_pretrained_cnn_feature_models.items():
  print(name)

## 4.2. Setup base hyper-parameters for training


In [ ]:
# Set hyperparameters settings
batch_size = 32
num_epochs = 10
num_iterations = len(X_train/batch_size)
image_height = 224
image_width = 224
num_channels = 3 #rgb
optimizer = 'rmsprop'
patience = 20
verbose = 1
factor = 0.2
min_lr = 0.0001
augment_images = False
fine_tune_pre_trained_cnn_model = False
monitor = 'val_loss'

# Set custom metrics to monitor
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
]

# Set adaptive learing rate setting
adaptive_lr = ReduceLROnPlateau(monitor=monitor, 
                                patience=patience, 
                                verbose=verbose, 
                                factor=factor, 
                                min_lr=min_lr)

# Set model check-pointing settings
mcp = ModelCheckpoint('model.h5')

# Set early stop settings
es = EarlyStopping(verbose=verbose, patience=patience)

# Set tensorboard callbacks for log visualization
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

## 4.3. Setup on demand image augmentation
### Use customizable hyper-params
a. featurewise_center=False,  # set input mean to 0 over the dataset  
b. samplewise_center=False,  # set each sample mean to 0  
c. featurewise_std_normalization=False,  # divide inputs by std of the dataset  
d. samplewise_std_normalization=False,  # divide each input by its std  
e. zca_whitening=False,  # apply ZCA whitening  
f. rotation_range=10,  # rotate images in the range (degrees, 0 to 180)  
g. zoom_range = 0.1, # Randomly zoom image   
h. width_shift_range=0.1,  # randomly shift images horizontally  
i. height_shift_range=0.1,  # randomly shift images vertically  
j. horizontal_flip=True,  # randomly flip images  
k. vertical_flip=True)  # randomly flip images  

In [ ]:
# Set hyper params for image augmentation
featurewise_center=False 
samplewise_center=False 
featurewise_std_normalization=False 
samplewise_std_normalization=False 
zca_whitening=False 
rotation_range=20 
zoom_range = [1.0,1.2] 
width_shift_range=0.0
height_shift_range=0.0
horizontal_flip=True 
vertical_flip=True

In [ ]:
# Configure Image Data Generator
def get_augmented_images():
  gen = ImageDataGenerator(
        rotation_range = rotation_range,
        zoom_range = zoom_range,
        horizontal_flip = horizontal_flip,
        vertical_flip = vertical_flip)
  return gen

## 4.4. Build hybrid CNN + DNN classification model

### 1. Use pre-trained CNN model 
#### When used as a feature extractor  
a. Freeze weights of pre-trained CNN model  
b. Freeze layers of pre-trained CNN model  

### 2. Add custom layers   
a. Dense  
b. Dropout    - 0.2  
c. BatchNormalization  

### Note:
a. We added BatchNormalization layer based on reading from this [paper](https://ui.adsabs.harvard.edu/abs/2016arXiv161201452S/abstract) which showed better results for pre-trained CNN with batch normalization vs without.  

b. We experimented with and without BatchNormalization layer and observed BatchNormalization to outperform vs without BatchNormalization.  

### 3. Add DNN/RNN layer to classify  
a. Loss       - Binary cross entropy  
b. Optimizer  - rmsProp/sgd/adam  
c. Activation - softmax/sigmoid  

### 4. Use customizable hyper-params
a. batch_size  
b. num_epochs  
c. num_iterations = len(X_train/batch_size)  
d. patience  
e. adaptive lr  
f. model check pointing  
g. augment_images  


In [ ]:
from keras.layers import Lambda, Reshape
# train and evaluate pre-trained CNN models 
def train_evaluate_classifiers(X_train, y_train, X_test, y_test, 
                               augment_images, 
                               add_batch_normalization,
                               fine_tune_pre_trained_cnn_model):
  eval_metrics = {'model_name': [], 
                  'model_params': [], 
                  'history': [],
                  'training_accuracy': [],
                  'val_accuracy': [],
                  'model' : []
                  }

   # perform image augmentation if requested
  if (augment_images):
    train_gen = get_augmented_images()
    train_gen.fit(X_train)
    test_gen = get_augmented_images()
    test_gen.fit(X_test)
    
  # do a muller loop on the models
  for name, model in my_pretrained_cnn_feature_models.items():
    print('\n------- Begin Training Model = %s for %.2f epochs --------' %(name, num_epochs))

    # set input shape
    input_shape=(image_height, image_width, num_channels)
    
    # get pre-trained cnn feature model with max pool
    pre_trained_cnn_feature_model = model(include_top=False, 
                                    pooling='max', 
                                    input_shape=input_shape)

    # if fine tune pre-trained cnn model, set, last layer trainable
    if (fine_tune_pre_trained_cnn_model):
       # freeze weights of pre-trained cnn feature model
       pre_trained_cnn_feature_model.trainable = True

       # freeze all layers before layer 15, i.e make last 4 layers trainable
       for layer in pre_trained_cnn_feature_model.layers[:15]:
          layer.trainable = False
    else:
       # freeze weights of pre-trained cnn feature model
       pre_trained_cnn_feature_model.trainable = False

       # freeze layers of pre-trained cnn feature model
       for layer in pre_trained_cnn_feature_model.layers:
        layer.trainable = False
    
    # add custom layers over pre-trained model
    model = tf.keras.models.Sequential()
    model.add(pre_trained_cnn_feature_model)
    model.add(Dense(512, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))

    # add batch normalization on demand
    if (add_batch_normalization):
      model.add(layers.BatchNormalization())
       
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', 
                  metrics=METRICS, 
                  optimizer=optimizer)
    # fit
    if (augment_images):
      history=model.fit(x = train_gen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data = test_gen.flow(X_test, y_test),
                        verbose = 1, epochs = num_epochs,
                        callbacks=[adaptive_lr, mcp, es, tensorboard_callback])
    else:
      history=model.fit(X_train, y_train, batch_size=batch_size,
                        validation_data = (X_test, y_test),
                        verbose = 1, epochs = num_epochs,
                        callbacks=[adaptive_lr, mcp, es, tensorboard_callback])

    # calculate all relevant metrics
    eval_metrics['model_name'].append(name)
    eval_metrics['model_params'].append(pre_trained_cnn_feature_model.count_params())
    eval_metrics['history'].append(history)
    eval_metrics['training_accuracy'].append(history.history['accuracy'][-1])
    eval_metrics['val_accuracy'].append(history.history['val_accuracy'][-1])
    eval_metrics['model'].append(model)

    print('\n------- End Training Model = %s for %.2f epochs --------' %(name, num_epochs))
  return eval_metrics 

## 4.5. Fine tune hyper-parameters and evaluate best CNN + DNN classifier

In [ ]:
# fine tune and evaluate best pre-trained CNN models with DNN classifer
def hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                 augment_images,
                                                 add_batch_normalization,
                                                 dropout_rate,
                                                 optimizer,
                                                 num_epochs,
                                                 fine_tune_pre_trained_cnn_model):
  eval_metrics = {'model_name': [], 
                  'model_params': [], 
                  'history': [],
                  'training_accuracy': [],
                  'val_accuracy': [],
                  'model' : []
                  }

  pretrained_cnn_feature_models = {}
  model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}
  for name, model in tqdm(model_dictionary.items()):
     if ("VGG16" == name):
      pretrained_cnn_feature_models[name] = model
      break

  # perform image augmentation if requested
  if (augment_images):
    train_gen = get_augmented_images()
    train_gen.fit(X_train)
    test_gen = get_augmented_images()
    test_gen.fit(X_test)
    
  # do a muller loop on the models
  for name, model in pretrained_cnn_feature_models.items():
    print('\n------- Begin Training Model = %s for %.2f epochs --------' %(name, num_epochs))

    # set input shape
    input_shape=(image_height, image_width, num_channels)
    
    # get pre-trained cnn feature model with max pool
    pre_trained_cnn_feature_model = model(include_top=False, 
                                    pooling='max', 
                                    input_shape=input_shape)

    # if fine tune pre-trained cnn model, set, last layer trainable
    if (fine_tune_pre_trained_cnn_model):
       # freeze weights of pre-trained cnn feature model
       pre_trained_cnn_feature_model.trainable = True

       # freeze all layers before layer 15, i.e make last 4 layers trainable
       for layer in pre_trained_cnn_feature_model.layers[:15]:
          layer.trainable = False
    else:
       # freeze weights of pre-trained cnn feature model
       pre_trained_cnn_feature_model.trainable = False

       # freeze layers of pre-trained cnn feature model
       for layer in pre_trained_cnn_feature_model.layers:
        layer.trainable = False
    
    # Make sure you have frozen the correct layers
    for i, layer in enumerate(pre_trained_cnn_feature_model.layers):
      print(i, layer.name, layer.trainable)

    # add custom layers over pre-trained model
    model = tf.keras.models.Sequential()
    model.add(pre_trained_cnn_feature_model)
    model.add(Dense(512, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate))

    # add batch normalization on demand
    if (add_batch_normalization):
      model.add(layers.BatchNormalization())
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', 
                  metrics=METRICS, 
                  optimizer=optimizer)
     
    # fit
    if (augment_images):
      history=model.fit(x = train_gen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data = test_gen.flow(X_test, y_test),
                        verbose = 1, epochs = num_epochs,
                        callbacks=[adaptive_lr, mcp, es, tensorboard_callback])
    else:
      history=model.fit(X_train, y_train, batch_size=batch_size,
                        validation_data = (X_test, y_test),
                        verbose = 1, epochs = num_epochs,
                        callbacks=[adaptive_lr, mcp, es, tensorboard_callback])

    # calculate all relevant metrics
    eval_metrics['model_name'].append(name)
    eval_metrics['model_params'].append(pre_trained_cnn_feature_model.count_params())
    eval_metrics['history'].append(history)
    eval_metrics['training_accuracy'].append(history.history['accuracy'][-1])
    eval_metrics['val_accuracy'].append(history.history['val_accuracy'][-1])
    eval_metrics['model'].append(model)

    print('\n------- End Training Model = %s for %.2f epochs --------' %(name, num_epochs))
  return eval_metrics 

## 4.6. Fine tune hyper-parameters and evaluate best CNN + LSTM classifier

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Embedding
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM

# fine tune and evaluate best pre-trained CNN models with DNN classifer
def hyper_param_tune_evaluate_best_RNN_classifer(X_train, y_train, X_test, y_test, 
                                                 augment_images,
                                                 add_batch_normalization,
                                                 dropout_rate,
                                                 optimizer,
                                                 num_epochs,
                                                 num_embeddings,
                                                 lstm_units,
                                                 add_dropout,
                                                 fine_tune_pre_trained_cnn_model):
  eval_metrics = {'model_name': [], 
                  'model_params': [], 
                  'history': [],
                  'training_accuracy': [],
                  'val_accuracy': [],
                  'model' : []
                  }

  # perform image augmentation if requested
  if (augment_images):
    train_gen = get_augmented_images()
    train_gen.fit(X_train)
    test_gen = get_augmented_images()
    test_gen.fit(X_test)
    
  # do a muller loop on the models
  name = 'VGG16'
  print('\n------- Begin Training Model = %s for %.2f epochs --------' %(name, num_epochs))

  # set input shape
  input_shape=(image_height, image_width, num_channels)

  pre_trained_cnn_feature_model = VGG16(include_top=False, 
                                       input_shape=input_shape, 
                                       weights="imagenet") 

  # if fine tune pre-trained cnn model, set, last layer trainable
  if (fine_tune_pre_trained_cnn_model):
    # freeze weights of pre-trained cnn feature model
    pre_trained_cnn_feature_model.trainable = True

    # freeze all layers before layer 15, i.e make last 4 layers trainable
    for layer in pre_trained_cnn_feature_model.layers[:15]:
      layer.trainable = False
  else:
    # freeze weights of pre-trained cnn feature model
    pre_trained_cnn_feature_model.trainable = False

    # freeze layers of pre-trained cnn feature model
    for layer in pre_trained_cnn_feature_model.layers:
      layer.trainable = False
    
  # Make sure you have frozen the correct layers
  for i, layer in enumerate(pre_trained_cnn_feature_model.layers):
    print(i, layer.name, layer.trainable)

  # Get CNN features
  features = GlobalAveragePooling2D()(pre_trained_cnn_feature_model.output)

  # Add embeddings, LSTM on top 
  embed_layer = Embedding(num_embeddings, lstm_units, mask_zero=True)(features)
  lstm_layer = LSTM(lstm_units)(embed_layer)

  # Add custom layers
  hidden_layer = Dense(512, activation="relu")(lstm_layer)
  if (add_dropout):
    hidden_layer = Dropout(dropout_rate)(hidden_layer)
  hidden_layer = Dense(128, activation="relu")(hidden_layer)

  if (add_dropout):
    hidden_layer = Dropout(dropout_rate)(hidden_layer)

  if (add_batch_normalization):
    hidden_layer = BatchNormalization() (hidden_layer)
  
  outputs = Dense(2, activation="softmax")(hidden_layer)
  model = Model([pre_trained_cnn_feature_model.input], outputs)
  model.compile(loss='binary_crossentropy', 
                metrics=METRICS, 
                optimizer=optimizer)
  model.summary()
     
  # fit
  if (augment_images):
    history=model.fit(x = train_gen.flow(X_train, y_train, batch_size=batch_size),
                     validation_data = test_gen.flow(X_test, y_test),
                     verbose = 1, epochs = num_epochs,
                     callbacks=[adaptive_lr, mcp, es, tensorboard_callback])
  else:
    history=model.fit(X_train, y_train, batch_size=batch_size,
                      validation_data = (X_test, y_test),
                      verbose = 1, epochs = num_epochs,
                      callbacks=[adaptive_lr, mcp, es, tensorboard_callback])

  # calculate all relevant metrics
  eval_metrics['model_name'].append(name)
  eval_metrics['model_params'].append(pre_trained_cnn_feature_model.count_params())
  eval_metrics['history'].append(history)
  eval_metrics['training_accuracy'].append(history.history['accuracy'][-1])
  eval_metrics['val_accuracy'].append(history.history['val_accuracy'][-1])
  eval_metrics['model'].append(model)

  print('\n------- End Training Model = %s for %.2f epochs --------' %(name, num_epochs))
  model.summary()
  return eval_metrics 

## 4.7. Setup model for training with CNN + custom layers + DNN layers¶


In [ ]:
# Train the different models
def train_model(augment_images, add_batch_normalization, fine_tune_pre_trained_cnn_model):
  warnings.filterwarnings('ignore')
  eval_metrics = train_evaluate_classifiers(X_train, y_train, X_val, y_val, 
                                            augment_images,
                                            add_batch_normalization,
                                            fine_tune_pre_trained_cnn_model)
  return eval_metrics

# 5.0. Visualization Helpers for classification

## 5.1. Helper - Visualize models by accuracy of prediction

In [ ]:
# Plot accuracy results
def visualize_accuracy_results(metrics):
  print('----- Displaying Models by accuracy of prediction -----')
  fig, ax = plt.subplots(ncols=7, figsize=(35,5))
  
  # Columns = model_name	model_params	history	test_accuracy	model
  for i, row in metrics.iterrows():
    model_name = row['model_name']
    history = row['history']
    ax[i].set_title(model_name)
    ax[i].plot(history.history["accuracy"], 'b', label="training accuracy")
    ax[i].plot(history.history["val_accuracy"], 'r', label="validation vccuracy")
    ax[i].legend(loc="best")
    ax[i].grid()
  plt.show()

## 5.2. Helper - Visualize models by size =(model params)


In [ ]:
# Report results
def visualize_models_by_size(metrics):
  print('----- Displaying Models by size=(model params) -----')
  metrics.sort_values('model_params', inplace=True) 
  display(metrics[['model_name', 'model_params', 'training_accuracy', 'val_accuracy']])

## 5.3. Helper - Visualize models by accuracy of prediction vs size

In [ ]:
# Visualize accuracy of different models
def visualize_accuracy_vs_size(metrics):
  print('----- Displaying Models by accuracy vs size -----')
  markers=[".",",","o","v","^","<",">","1","2","3","4","8","s","p","P","*","h","H","+","x","X","D","d","|","_",4,5,6,7,8,9,10,11]

  # Plot metrics
  plt.figure(figsize=(10,8))
  for model in metrics.itertuples():
    plt.scatter(model.model_params, model.val_accuracy, 
                label=model.model_name, marker=markers[model.Index], 
                s=150, linewidths=2)
  plt.xscale('log')
  plt.xlabel('Total Parameters in Model')
  plt.ylabel('Validation Accuracy with 25 epochs')
  plt.ylabel('Validation Accuracy with %s epochs' %(num_epochs))
  plt.title('Accuracy vs Model Params')
  plt.legend(bbox_to_anchor=(1, 1), loc='upper left'); 

## 5.4. Helper - Visualize models by loss 

In [ ]:
# Plot loss results
def visualize_loss_results(metrics):
  print('----- Displaying Models by loss -----')
  fig, ax = plt.subplots(ncols=7, figsize=(35,5))

  # Columns = model_name	model_params	history	test_accuracy	model
  for i, row in metrics.iterrows():
    model_name = row['model_name']
    history = row['history']
    ax[i].set_title(model_name)
    ax[i].plot(history.history["loss"], 'b', label="training loss")
    ax[i].plot(history.history["val_loss"], 'r', label="validation loss")
    ax[i].legend(loc="best")
    ax[i].grid()

## 5.5. Helper - Visualize models by AUC

In [ ]:
# Plot auc results
def visualize_auc_results(metrics):
  print('----- Displaying Models by auc -----')
  fig, ax = plt.subplots(ncols=7, figsize=(35,5))

  # Columns = model_name	model_params	history	test_accuracy	model
  for i, row in metrics.iterrows():
    model_name = row['model_name']
    history = row['history']
    ax[i].set_title(model_name)
    ax[i].plot(history.history["auc"], 'b', label="AUC")
    ax[i].legend(loc="best")
    ax[i].grid()

## 5.6. Helper - Visualize models by precision 

In [ ]:
# Plot auc results
def visualize_precision_results(metrics):
  print('----- Displaying Models by precision -----')
  fig, ax = plt.subplots(ncols=7, figsize=(35,5))

  # Columns = model_name	model_params	history	test_accuracy	model
  for i, row in metrics.iterrows():
    model_name = row['model_name']
    history = row['history']
    ax[i].set_title(model_name)
    ax[i].plot(history.history["precision"], 'b', label="Training Precision")
    ax[i].plot(history.history["val_precision"], 'r', label="Validation Precision")
    ax[i].legend(loc="best")
    ax[i].grid()

## 5.7. Helper - Visualize models by recall

In [ ]:
# Plot auc results
def visualize_recall_results(metrics):
  print('----- Displaying Models by recall -----')
  fig, ax = plt.subplots(ncols=7, figsize=(35,5))

  # Columns = model_name	model_params	history	test_accuracy	model
  for i, row in metrics.iterrows():
    model_name = row['model_name']
    history = row['history']
    ax[i].set_title(model_name)
    ax[i].plot(history.history["recall"], 'b', label="Training Recall")
    ax[i].plot(history.history["val_recall"], 'r', label="Validation Recall")
    ax[i].legend(loc="best")
    ax[i].grid()

## 5.8. Helper - Visualize best model summary 

In [ ]:
# Report summary of best model
def visualize_best_model_summary(metrics):
  print('----- Displaying Best Model summary -----')
  best_model = 0
  metrics.sort_values(by='val_accuracy', ascending=False, inplace=True)
  best_model_test_accuracy = metrics.iloc[0]['val_accuracy']

  # get the best model, report summary
  for model in metrics.itertuples():
    if model.val_accuracy == best_model_test_accuracy:
      best_model = model.model
      print('Best model for breast cancer image classification is %s with validation accuracy of %0.02f percent.' %(model.model_name, model.val_accuracy*100))
      
      # report best model summary
      best_model.summary()
      break

  return best_model

## 5.9. Helper - Visualize best model classification report

In [ ]:
# Visualize classification report
def visualize_best_model_classification_report(model):
  y_pred = model.predict(X_test)
  y_predicted = np.argmax(y_pred, axis=1)
  y_expected = np.argmax(y_test, axis=1)
  report = classification_report(y_true=y_expected, y_pred=y_predicted, target_names=['negative', 'positive'])
  print(report)

## 5.10. Helper - Visualize best model confusion matrix

In [ ]:
# plot confusion matrics
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Visualize confusion matrix
def visualize_best_model_confusion_matrix(model):
  y_pred = model.predict(X_test)
  y_predicted = np.argmax(y_pred, axis=1)
  y_expected = np.argmax(y_test, axis=1)
  cm = confusion_matrix(y_expected, y_predicted)
  cm_plot_label =['Benign', 'Malignant']
  plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Matrix')

## 5.11. Helper - Visualize best model top common errors in prediction

In [ ]:
# visualize top commonprediction errors
def visualize_best_model_top_common_errors(model):
  y_pred = best_model_CNN_with_DNN.predict(X_test)
  y_predicted = np.argmax(y_pred, axis=1)
  y_expected = np.argmax(y_test, axis=1)
  i=0
  prop_class=[]
  mis_class=[]

  for i in range(len(Y_test)):
      if(y_expected[i] != y_predicted[i]):
            prop_class.append(i)
      if(len(prop_class)==8):
            break
        
  i=0
  for i in range(len(Y_test)):
      if(not y_expected[i]== y_predicted[i]):
          mis_class.append(i)
      if(len(mis_class)==8):
          break

  # # Display first 8 images of benign
  w=60
  h=40
  fig=plt.figure(figsize=(18, 10))
  columns = 4
  rows = 2

  def Transfername(namecode):
      if namecode==1:
          return "Malignant"
      else:
          return "Benign"
      
  for i in range(len(prop_class)):
      ax = fig.add_subplot(rows, columns, i+1)
      ax.set_title("Predicted result:"+ Transfername(y_predicted[prop_class[i]])
                        +"\n"+"Actual result: "+ Transfername(y_expected[prop_class[i]]))
      plt.imshow(X_test[prop_class[i]], interpolation='nearest')
  plt.show()

# 6.0. Train and Evaluate multiple CNN + custom layers + DNN classifiers

## 6.1. Train classifiers with base hyper-param settings

In [ ]:
# train
augment_images = False
add_batch_normalization = False
fine_tune_pre_trained_cnn_model = False

eval_metrics_CNN_with_DNN = train_model(augment_images, 
                                        add_batch_normalization, 
                                        fine_tune_pre_trained_cnn_model)
df_eval_metrics_CNN_with_DNN = pd.DataFrame(eval_metrics_CNN_with_DNN)

## 6.2. Evaluate classifiers by accuracy of prediction

In [ ]:
# Evaluate model results
visualize_accuracy_results(df_eval_metrics_CNN_with_DNN)

## 6.2. Evaluate classifiers by size 

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_metrics_CNN_with_DNN)

## 6.3. Evaluate classifers by accuracy vs size

In [ ]:
# Visualize models by accuracy vs size
visualize_accuracy_vs_size(df_eval_metrics_CNN_with_DNN)

## 6.4. Evaluate classifers by loss

In [ ]:
# Visualize models by loss 
visualize_loss_results(df_eval_metrics_CNN_with_DNN)

## 6.5. Evaluate classifiers by AUC

In [ ]:
# Visualize models by AUC
visualize_auc_results(df_eval_metrics_CNN_with_DNN)

## 6.6. Evaluate classifers by precision

In [ ]:
# Visualize models by precision
visualize_precision_results(df_eval_metrics_CNN_with_DNN)

## 6.7. Evaluate classifers by recall

In [ ]:
# Visualize models by recall
visualize_recall_results(df_eval_metrics_CNN_with_DNN)

# 7.0. Fine tune best CNN + custom layers + DNN classifer hyper parameters

## 7.1. Fine tune hyper parameters (experiment - 1) num_epochs
1. Use baseline hyper-parameters, set num_epochs=25

In [ ]:
# train
num_epochs = 20
augment_images = False
add_batch_normalization = False
fine_tune_pre_trained_cnn_model = False
dropout_rate = 0.2
optimizer = 'rmsprop'
eval_tuned_metrics_CNN_with_DNN_1 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_1 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_1)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_1)

## 7.2. Fine tune hyper parameters (experiment - 2) batch normalization
1. Use baseline hyper-parameters, set num_epochs=25
2. Add batch normalization

In [ ]:
# train
augment_images = False
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = False
num_epochs = 20
dropout_rate = 0.2
optimizer = 'rmsprop'
eval_tuned_metrics_CNN_with_DNN_2 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_2 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_2)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_2)

## 7.3. Fine tune hyper parameters (experiment - 3) image augmentation
1. Use baseline hyper-parameters, set num_epochs=25
2. Add batch normalization
3. Add image augmentation

In [ ]:
# train
augment_images = True
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = False
num_epochs = 20
dropout_rate = 0.2
optimizer = 'rmsprop'
eval_tuned_metrics_CNN_with_DNN_3 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_3 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_3)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_3)

## 7.4. Fine tune hyper parameters (experiment - 4) dropout_rate
1. Use baseline hyper-parameters, set num_epochs=25
2. Add batch normalization
3. Add image augmentation
4. dropout_rate = 0.5

In [ ]:
# train
augment_images = True
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = False
num_epochs = 20
dropout_rate = 0.5
optimizer = 'rmsprop'
eval_tuned_metrics_CNN_with_DNN_4 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_4 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_4)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_4)

## 7.5. Fine tune hyper parameters (experiment - 5) optimizer
1. Use baseline hyper-parameters, set num_epochs=25
2. Add batch normalization
3. Add image augmentation
4. dropout_rate = 0.5
5. optimizer =adam

In [ ]:
# train
augment_images = True
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = False
num_epochs = 20
dropout_rate = 0.5
optimizer = 'adam'
eval_tuned_metrics_CNN_with_DNN_5 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_5 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_5)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_5)

## 7.6. Fine tune hyper parameters (experiment - 6) unfreeze pre-trained layers
1. Use baseline hyper-parameters, set num_epochs=25
2. Add batch normalization
3. Add image augmentation
4. dropout_rate = 0.5
5. optimizer=rmsprop
6. Unfreeze pre-trained CNN, fine tune last layer

In [ ]:
# train
augment_images = True
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = True
num_epochs = 20
dropout_rate = 0.5
optimizer = 'rmsprop'
eval_tuned_metrics_CNN_with_DNN_6 = hyper_param_tune_evaluate_best_DNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_DNN_6 = pd.DataFrame(eval_tuned_metrics_CNN_with_DNN_6)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_DNN_6)

# 8.0. Evaluate best CNN + custom layers + DNN classifer metrics

## 8.1. Evaluate best classifier model summary

In [ ]:
# Visualize best model summary
best_model_CNN_with_DNN = visualize_best_model_summary(df_eval_tuned_metrics_CNN_with_DNN_6)

## 8.2. Evaluate best classifer classification report

In [ ]:
# Visualize best model classification results
visualize_best_model_classification_report(best_model_CNN_with_DNN)

## 8.3. Evaluate best classifer confusion matrix

In [ ]:
# Visualize best model confusion matrix
visualize_best_model_confusion_matrix(best_model_CNN_with_DNN)

## 8.4. Evaluate best classifer top common errors in prediction

In [ ]:
# Visualize best model top common errors in prediction
visualize_best_model_top_common_errors(best_model_CNN_with_DNN)

# 9.0.Fine tune best CNN + custom layers + LSTM classifier hyper parameters

## 9.1. Fine tune hyper parameters (experiment - 1) best DNN classifier settings
1. num_epochs = 20  
2. augment_images  
3. add batch normalization layer
4. fine tune pre-trained CNN model  
5. dropout_rate  = 0.5  
6. optimizer = rmsprop  



In [ ]:
# train
num_epochs = 20
augment_images = True
add_batch_normalization = True
fine_tune_pre_trained_cnn_model = True
dropout_rate = 0.5
optimizer = 'rmsprop'
num_embeddings = 512
lstm_units = 64
add_dropout = True
eval_tuned_metrics_CNN_with_RNN_1 = hyper_param_tune_evaluate_best_RNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 num_embeddings,
                                                                                 lstm_units,
                                                                                 add_dropout,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_RNN_1 = pd.DataFrame(eval_tuned_metrics_CNN_with_RNN_1)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_RNN_1)

## 9.2. Fine tune hyper parameters (experiment - 2) custom
1. Increase lstm units to 512
2. Turn off drop out 
3. Turn off batch normalization
4. Turn off fine tuning of pre-trained CNN model

In [ ]:
# train
num_epochs = 20
augment_images = True
add_batch_normalization = False
fine_tune_pre_trained_cnn_model = False
dropout_rate = 0.5
optimizer = 'rmsprop'
num_embeddings = 512
lstm_units = 512
add_dropout = False
eval_tuned_metrics_CNN_with_RNN_2 = hyper_param_tune_evaluate_best_RNN_classifer(X_train, y_train, X_test, y_test, 
                                                                                 augment_images,
                                                                                 add_batch_normalization,
                                                                                 dropout_rate,
                                                                                 optimizer,
                                                                                 num_epochs,
                                                                                 num_embeddings,
                                                                                 lstm_units,
                                                                                 add_dropout,
                                                                                 fine_tune_pre_trained_cnn_model)
df_eval_tuned_metrics_CNN_with_RNN_2 = pd.DataFrame(eval_tuned_metrics_CNN_with_RNN_2)

In [ ]:
# Visualize models by size
visualize_models_by_size(df_eval_tuned_metrics_CNN_with_RNN_2)

# 10.0. Evaluate best CNN + custom layers + LSTM classifier metrics

## 10.1. Evaluate best classifier model summary

In [ ]:
# Visualize best model summary
best_model_CNN_with_RNN = visualize_best_model_summary(df_eval_tuned_metrics_CNN_with_RNN_1)

## 10.2. Evaluate best classifier classification report

In [ ]:
# Visualize best model classification results
visualize_best_model_classification_report(best_model_CNN_with_RNN)

## 10.3. Evaluate best classifier confusion matrix

In [ ]:
# Visualize best model confusion matrix
visualize_best_model_confusion_matrix(best_model_CNN_with_RNN)

## 10.4. Evaluate best classifer top common errors in predicition

In [ ]:
# Visualize best model top common errors in prediction
visualize_best_model_top_common_errors(best_model_CNN_with_RNN)

# 11.0. Tensorboard integration 

In [ ]:
# Bring up tensorboard
%load_ext tensorboard
%tensorboard --logdir logs

# 12.0. Best Model checkpoint and serve using tensorflow serving

The professor provided this [notebook](https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/serving/rest_simple.ipynb) to be used as reference for using tensorflow serving inside google colab.

## 12.1. Checkpoint and Serve best CNN with DNN model

### 12.1.1. Checkpoint

In [ ]:
best_model = best_model_CNN_with_DNN

In [ ]:
# Save best model
MODEL_DIR = tempfile.gettempdir()
version = 1
model = best_model
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

In [ ]:
# Inspect saved model
!saved_model_cli show --dir {export_path} --all

### 12.1.2. Inspect tensorflow serving

In [ ]:
# Install grpcio
!pip install -Uq grpcio==1.32.0
print('TensorFlow version: {}'.format(tf.__version__))

In [ ]:
# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.32.0

print('TensorFlow version: {}'.format(tf.__version__))

In [ ]:
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

### 12.1.3. Install tensorflow model server if needed

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

### 12.1.4. Send requests to model using tensorflow serving

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=breast_cancer_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
test_images=X_test
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:10].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

### 12.1.5. Make REST requests to server, test predictions

In [ ]:
!pip install -q requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/breast_cancer_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(len(test_images[1:3])):
  print(np.argmax(predictions[i]),y_test[i])

# 13.0. Archive best model, logs

In [ ]:
shutil.make_archive('saved_model_tfs', 'zip', MODEL_DIR)
shutil.make_archive('logs', 'zip', '/content/logs')